# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 14 2022 10:35AM,241856,19,60017377,"GUSA Granules USA, Inc.",Released
1,Jul 14 2022 10:30AM,241854,10,8273920,Beach Products Inc,Released
2,Jul 14 2022 10:30AM,241854,10,8273921,Beach Products Inc,Released
3,Jul 14 2022 10:30AM,241854,10,8273922,Beach Products Inc,Released
4,Jul 14 2022 10:30AM,241854,10,8273923,Beach Products Inc,Released
5,Jul 14 2022 10:30AM,241854,10,8273924,Beach Products Inc,Released
6,Jul 14 2022 10:30AM,241854,10,8273925,Beach Products Inc,Released
7,Jul 14 2022 10:30AM,241854,10,8273926,Beach Products Inc,Released
8,Jul 14 2022 10:30AM,241854,10,8273927,Beach Products Inc,Released
9,Jul 14 2022 10:30AM,241854,10,8273928,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,241851,Executing,1
37,241852,Released,1
38,241853,Released,19
39,241854,Released,14
40,241856,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241851,NaN,1.0,NaN
241852,NaN,NaN,1.0
241853,NaN,NaN,19.0
241854,NaN,NaN,14.0
241856,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241758,0.0,1.0,0.0
241759,0.0,1.0,0.0
241771,0.0,1.0,0.0
241774,0.0,0.0,1.0
241784,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241758,0,1,0
241759,0,1,0
241771,0,1,0
241774,0,0,1
241784,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241758,0,1,0
1,241759,0,1,0
2,241771,0,1,0
3,241774,0,0,1
4,241784,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241758,,1,
1,241759,,1,
2,241771,,1,
3,241774,,,1
4,241784,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc."
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc
15,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.
34,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC"
35,Jul 14 2022 9:32AM,241851,10,Emerginnova
36,Jul 14 2022 9:30AM,241850,20,"Exact-Rx, Inc."
39,Jul 14 2022 9:30AM,241849,12,Hush Hush
41,Jul 14 2022 9:29AM,241848,12,Hush Hush
79,Jul 14 2022 9:25AM,241847,10,Bio-PRF
80,Jul 14 2022 9:20AM,241846,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",,,1
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,,,14
2,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,,,19
3,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC",,,1
4,Jul 14 2022 9:32AM,241851,10,Emerginnova,,1,
5,Jul 14 2022 9:30AM,241850,20,"Exact-Rx, Inc.",,,3
6,Jul 14 2022 9:30AM,241849,12,Hush Hush,,,2
7,Jul 14 2022 9:29AM,241848,12,Hush Hush,,6,32
8,Jul 14 2022 9:25AM,241847,10,Bio-PRF,,,1
9,Jul 14 2022 9:20AM,241846,20,Alumier Labs Inc.,,,25


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",1,,
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,14,,
2,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,19,,
3,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC",1,,
4,Jul 14 2022 9:32AM,241851,10,Emerginnova,,1,
5,Jul 14 2022 9:30AM,241850,20,"Exact-Rx, Inc.",3,,
6,Jul 14 2022 9:30AM,241849,12,Hush Hush,2,,
7,Jul 14 2022 9:29AM,241848,12,Hush Hush,32,6,
8,Jul 14 2022 9:25AM,241847,10,Bio-PRF,1,,
9,Jul 14 2022 9:20AM,241846,20,Alumier Labs Inc.,25,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",1,,
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,14,,
2,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,19,,
3,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC",1,,
4,Jul 14 2022 9:32AM,241851,10,Emerginnova,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,14.0,NaN,NaN
2,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,19.0,NaN,NaN
3,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Jul 14 2022 9:32AM,241851,10,Emerginnova,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,14.0,0.0,0.0
2,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,19.0,0.0,0.0
3,Jul 14 2022 9:48AM,241852,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Jul 14 2022 9:32AM,241851,10,Emerginnova,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3385678,173.0,1.0,0.0
12,725468,34.0,7.0,0.0
16,1934381,5.0,4.0,0.0
19,725493,36.0,18.0,10.0
20,725481,28.0,1.0,0.0
21,1450828,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3385678,173.0,1.0,0.0
1,12,725468,34.0,7.0,0.0
2,16,1934381,5.0,4.0,0.0
3,19,725493,36.0,18.0,10.0
4,20,725481,28.0,1.0,0.0
5,21,1450828,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,173.0,1.0,0.0
1,12,34.0,7.0,0.0
2,16,5.0,4.0,0.0
3,19,36.0,18.0,10.0
4,20,28.0,1.0,0.0
5,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,173.0
1,12,Released,34.0
2,16,Released,5.0
3,19,Released,36.0
4,20,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,10.0,0.0,0.0
Executing,1.0,7.0,4.0,18.0,1.0,1.0
Released,173.0,34.0,5.0,36.0,28.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,10.0,0.0,0.0
1,Executing,1.0,7.0,4.0,18.0,1.0,1.0
2,Released,173.0,34.0,5.0,36.0,28.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,10.0,0.0,0.0
1,Executing,1.0,7.0,4.0,18.0,1.0,1.0
2,Released,173.0,34.0,5.0,36.0,28.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()